In [3]:
import os
import subprocess

def merge_videos_and_create_timestamps(folder_path):
    """
    Merges all MP4 videos in a folder into a single video and creates a timestamp file in HH:MM:SS format.

    Args:
        folder_path (str): The path to the folder containing the MP4 videos.
    """
    if not os.path.exists(folder_path):
        print(f"Error: The folder path '{folder_path}' does not exist.")
        return

    # List to store video files and timestamps
    video_files = []
    timestamps = []
    total_duration = 0

    # Find all MP4 files in the folder (case-insensitive match for extensions)
    for filename in sorted(os.listdir(folder_path)):  # Sort by name for consistent ordering
        if filename.lower().endswith(".mp4"):
            filepath = os.path.join(folder_path, filename)
            video_files.append(filepath)

            # Get video duration using ffprobe
            try:
                probe = subprocess.run(
                    ["ffprobe", "-v", "error", "-show_entries", "format=duration", "-of", "default=noprint_wrappers=1:nokey=1", filepath],
                    stdout=subprocess.PIPE,
                    stderr=subprocess.PIPE,
                    text=True,
                )
                duration = float(probe.stdout.strip())
                timestamps.append((filename, total_duration))
                total_duration += duration
            except Exception as e:
                print(f"Error reading duration for file '{filename}': {e}")

    # Check if there are videos to merge
    if not video_files:
        print(f"No MP4 files found in the folder: {folder_path}")
        return

    # Dynamically create the merged video name from the last 4 folders
    folder_parts = os.path.normpath(folder_path).split(os.sep)
    if len(folder_parts) >= 4:
        merged_video_name = " - ".join(folder_parts[-4:]) + ".mp4"
    else:
        merged_video_name = "merged_video.mp4"  # Fallback name if the path is too short

    # Prepare an intermediate file list for FFmpeg concatenation
    file_list_path = os.path.join(folder_path, "file_list.txt")
    with open(file_list_path, "w", encoding="utf-8") as f:
        for video_file in video_files:
            f.write(f"file '{video_file.replace('\\', '/')}'\n")  # Fix path formatting

    # Merge videos using FFmpeg
    output_path = os.path.join(folder_path, merged_video_name)
    try:
        print("Merging videos...")
        subprocess.run(
            ["ffmpeg", "-f", "concat", "-safe", "0", "-i", file_list_path, "-c", "copy", output_path],
            check=True,
            text=True
        )
    except subprocess.CalledProcessError as e:
        print(f"FFmpeg error during merging: {e}")
        return

    # Write timestamps to a text file in HH:MM:SS format
    timestamp_file_path = os.path.join(folder_path, "timestamps.txt")
    with open(timestamp_file_path, "w", encoding="utf-8") as f:
        for filename, timestamp in timestamps:
            hours = int(timestamp // 3600)
            minutes = int((timestamp % 3600) // 60)
            seconds = int(timestamp % 60)
            f.write(f"{filename} - {hours:02d}:{minutes:02d}:{seconds:02d}\n")

    # Notify user
    print(f"Merged video saved to: {output_path}")
    print(f"Timestamps saved to: {timestamp_file_path}")

if __name__ == "__main__":
    # Get folder path from user input
    folder_path = input("Enter the folder path containing the videos: ").strip()

    # Merge videos and generate timestamps
    merge_videos_and_create_timestamps(folder_path)


Merging videos...
Merged video saved to: H:\Projects Control (PC)\10 Backup\05 Tutorials\Adobe\Illustrator\Udemy\UI And Web Design Using Adobe Illustrator 2017\Adobe - Illustrator - Udemy - UI And Web Design Using Adobe Illustrator 2017.mp4
Timestamps saved to: H:\Projects Control (PC)\10 Backup\05 Tutorials\Adobe\Illustrator\Udemy\UI And Web Design Using Adobe Illustrator 2017\timestamps.txt
